## Self Learning Notes
- Following came from the tutorial [Tabular Training](https://docs.fast.ai/tutorial.tabular.html)

In [1]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/Users/rumpagiri/.fastai/data/adult_sample/adult.csv'),Path('/Users/rumpagiri/.fastai/data/adult_sample/export.pkl'),Path('/Users/rumpagiri/.fastai/data/adult_sample/models')]

In [2]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [4]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

/Users/rumpagiri/Projects/fastai-medical-ml/myenv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [5]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [6]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

/Users/rumpagiri/Projects/fastai-medical-ml/myenv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [7]:
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
26935,5,10,5,13,5,5,1,-0.043765,-1.477090,1.148950
25129,7,9,1,8,2,5,1,-0.630961,-1.610586,0.362906


In [8]:
dls = to.dataloaders(bs=64)

In [9]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Married-civ-spouse,#na#,Husband,White,False,42.000000,136418.999046,9.0,>=50k
1,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Black,False,34.000000,297094.005541,9.0,<50k
2,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,False,34.000000,189809.000000,10.0,<50k
3,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,False,30.000000,116138.002373,13.0,>=50k
4,Private,Bachelors,Never-married,Other-service,Not-in-family,White,False,48.000000,267912.000120,13.0,>=50k
5,Private,Some-college,Divorced,Adm-clerical,Unmarried,White,False,38.000000,179262.000354,10.0,<50k
6,Private,9th,Never-married,Other-service,Unmarried,White,False,20.000001,447488.010749,5.0,<50k
7,Private,Some-college,Never-married,Machine-op-inspct,Not-in-family,White,False,39.000000,435637.989161,10.0,<50k
8,Private,Some-college,Married-civ-spouse,Adm-clerical,Wife,White,False,37.000000,93716.999510,10.0,>=50k
9,Private,Some-college,Married-civ-spouse,Transport-moving,Husband,White,False,32.000000,329993.006042,10.0,<50k


In [10]:
learn = tabular_learner(dls, metrics=accuracy)

In [11]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.360233,0.359637,0.840295,00:02


In [12]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,12.0,7.0,7.0,2.0,5.0,1.0,1.571022,-0.105441,-0.423137,0.0,0.0
1,5.0,15.0,5.0,11.0,2.0,5.0,1.0,-0.410762,-0.839926,1.934993,1.0,0.0
2,5.0,10.0,5.0,13.0,2.0,5.0,1.0,-0.557561,0.487990,1.148950,0.0,0.0
3,5.0,16.0,5.0,5.0,2.0,5.0,1.0,-0.630961,-0.751805,-0.030115,0.0,0.0
4,5.0,6.0,7.0,9.0,2.0,5.0,1.0,2.011419,-0.136114,-2.388245,0.0,0.0
5,5.0,16.0,1.0,11.0,5.0,5.0,1.0,-0.630961,-1.236912,-0.030115,0.0,0.0
6,5.0,10.0,5.0,11.0,5.0,5.0,1.0,-1.071357,-0.018704,1.148950,0.0,0.0
7,5.0,12.0,3.0,0.0,1.0,5.0,2.0,-0.704360,0.509805,-0.030115,1.0,0.0
8,5.0,12.0,3.0,0.0,1.0,5.0,1.0,0.910427,-1.220087,-0.423137,1.0,0.0


In [13]:
row, clas, probs = learn.predict(df.iloc[0])

/Users/rumpagiri/Projects/fastai-medical-ml/myenv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [14]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.001696,12.0,<50k


In [15]:
clas, probs

(tensor(0), tensor([0.6475, 0.3525]))

In [16]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

/Users/rumpagiri/Projects/fastai-medical-ml/myenv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [17]:
learn.get_preds(dl=dl)

(tensor([[0.6475, 0.3525],
         [0.4439, 0.5561],
         [0.9338, 0.0662],
         ...,
         [0.5366, 0.4634],
         [0.7016, 0.2984],
         [0.6555, 0.3445]]),
 None)

In [18]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
26935,5,10,5,13,5,5,1,-0.043765,-1.477090,1.148950
25129,7,9,1,8,2,5,1,-0.630961,-1.610586,0.362906
32342,6,16,3,5,1,5,1,2.084818,-0.069504,-0.030115


In [19]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()